In [2]:
from pandas import Series, DataFrame
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import csv
from IPython.display import display, HTML
import string
import re
pd.options.display.float_format = '${:10,.0f}'.format

In [4]:
# Account Assignment
def get_wb():
    wd_path = '../account_assignment/'
    wb_file = wd_path + 'FY17_WB_v1.xlsx'
    xl_wb = pd.ExcelFile(wb_file)
    df1 = xl_wb.parse('FY17 ACCOUNTS')
    df1.rename(columns=string.lower, inplace=True)
    fill_na = lambda x: np.nan
    df1['fy17 owner userid '] = df1['fy17 owner userid '].map(fill_na)
    df1['fy17 team agent name (for team agents only)'] = df1['fy17 team agent name (for team agents only)'].map(fill_na)
    df1['fy17 owner sl6 node'] = df1['fy17 owner sl6 node'].map(fill_na)
    xl_wb = pd.ExcelFile(wb_file)
    df2 = xl_wb.parse('NEW ACCOUNTS')
    df2.rename(columns=string.lower, inplace=True)
    return (df1, df2)
def get_rows_cols(df):
    return (len(df.index), len(df.columns))
def print_dim(rows, cols, comment=''):
    print comment + "Row(s): %d | Column(s): %d" % (rows, cols)
def read_get_wb_update(file_name):
    wd_path = '../account_assignment/'
    wb_file = wd_path + file_name
    xl_wb = pd.ExcelFile(wb_file)
    df1= xl_wb.parse('existing')
    df1.rename(columns=string.lower, inplace=True)
    df2= xl_wb.parse('NEW ACCOUNTS')
    df2.rename(columns=string.lower, inplace=True)
    return (df1, df2)
def compile_data(file_name, existing_cols, new_cols):
    # Prasad Kulkarni (prakulk2) <prakulk2@cisco.com>/Fri 6/24/2016 8:55 PM/FW: Accounts workbook for COMM FY17/Prakulk2 - SELECT ACCOUNT LIST -  FY17_WB_v1.xlsx
    # And df with prefix 'e' stands for existing, with prefix 'n' stands for new
    existing_cols = existing_cols.append(pd.Series(['updated_file_name']))
    new_cols = new_cols.append(pd.Series(['updated_file_name']))
    df_input = pd.read_excel('../account_assignment/'+file_name+'.xlsx')
    df_comp_existing = pd.DataFrame(columns=existing_cols)
    df_comp_new = pd.DataFrame(columns=new_cols)
    df_log = pd.DataFrame(columns=['file_name', 'description', 'existing_rows', 'existing_cols', 'new_rows', 'new_cols'])
    for index, input_data in df_input.iterrows():
        print 'Processing File %s...' % (input_data.file_name)
        #file_name = 'Prakulk2 - SELECT ACCOUNT LIST -  FY17_WB_v1.xlsx'
        (edf, ndf) = read_get_wb_update(input_data.file_name+'.xlsx')
        edf['updated_file_name'] = input_data.file_name
        ndf['updated_file_name'] = input_data.file_name
        (e_rows, e_cols) = get_rows_cols(edf)
        (n_rows, n_cols) = get_rows_cols(ndf)
        print_dim(e_rows, e_cols, comment='Existing Accounts=> ')
        print_dim(n_rows, n_cols, comment='New Accounts=> ')
        log_dict = {
            'file_name': [input_data.file_name],
            'description': [input_data.description],
            'existing_rows': [e_rows],
            'existing_cols': [e_cols],
            'new_rows': [n_rows],
            'new_cols': [n_cols]
        }
        df_comp_existing = pd.concat([df_comp_existing, edf])
        df_comp_new = pd.concat([df_comp_new, ndf])
        df_log = pd.concat([df_log, DataFrame.from_dict(log_dict)])
        print "Writing Individual data in Excel..."
        writer = pd.ExcelWriter('../account_assignment/compiled_'+input_data.file_name+'.xlsx', engine='xlsxwriter')
        edf.to_excel(writer, sheet_name='FY17 ACCOUNTS', columns=existing_cols)
        ndf.to_excel(writer, sheet_name='NEW ACCOUNTS', columns=new_cols)
        DataFrame.from_dict(log_dict).to_excel(writer, sheet_name='LOG')
        writer.save()
        print "Individual data has been written in Excel!"
        print 'End of Processing File %s!' % (input_data.file_name)
    print "Writing compiled data in Excel..."
    writer = pd.ExcelWriter('../account_assignment/assignment_fy17_accounts.xlsx', engine='xlsxwriter')
    df_comp_existing.to_excel(writer, sheet_name='FY17 ACCOUNTS', columns=existing_cols)
    df_comp_new.to_excel(writer, sheet_name='NEW ACCOUNTS', columns=new_cols)
    df_log.to_excel(writer, sheet_name='LOG')
    writer.save()
    print "Compiled data has been written in Excel!"
    print 'End of Compilation!'
    return (df_comp_existing, df_comp_new, df_log)

def validate_wb(df1, df2, df_comp_existing, df_comp_new):
    df1.loc[df1['row'].isin(df_comp_existing['row']), ['fy17 owner userid ', 'fy17 team agent name (for team agents only)', 'fy17 owner sl6 node']] = df_comp_existing[['fy17 owner userid ', 'fy17 team agent name (for team agents only)', 'fy17 owner sl6 node']]
    print "Writing validated data in Excel..."
    writer = pd.ExcelWriter('../account_assignment/assignment_fy17_accounts_validated.xlsx', engine='xlsxwriter')
    df1.to_excel(writer, sheet_name='FY17 ACCOUNTS', columns=existing_cols)
    df2.to_excel(writer, sheet_name='NEW ACCOUNTS', columns=new_cols)
    writer.save()
    print "Validated data has been written in Excel!"
    
def validate_wb2(df1, df2, df_comp_existing, df_comp_new):
    existing_cols = df1.columns
    new_cols = df2.columns
    existing_cols = existing_cols.append(pd.Series(['updated_file_name']))
    new_cols = new_cols.append(pd.Series(['updated_file_name']))
    print 'Mapping is going on...'
    for index, rows in df_comp_existing.iterrows():
        df1.ix[df1['row']==rows['row'], 'account type'] = rows['account type']
        df1.ix[df1['row']==rows['row'], 'fy17 owner userid '] = rows['fy17 owner userid ']
        df1.ix[df1['row']==rows['row'], 'fy17 team agent name (for team agents only)'] = rows['fy17 team agent name (for team agents only)']
        df1.ix[df1['row']==rows['row'], 'fy17 owner sl6 node'] = rows['fy17 owner sl6 node']
        df1.ix[df1['row']==rows['row'], 'updated_file_name'] = rows['updated_file_name']
    df2 = pd.concat([df2, df_comp_new])
    #df1.loc[df1['row'].isin(df_comp_existing['row']), ['fy17 owner userid ', 'fy17 team agent name (for team agents only)', 'fy17 owner sl6 node']] = df_comp_existing[['fy17 owner userid ', 'fy17 team agent name (for team agents only)', 'fy17 owner sl6 node']]
    print 'Mapping is complete!'
    print "Writing validated data in Excel..."
    writer = pd.ExcelWriter('../account_assignment/assignment_fy17_accounts_validated.xlsx', engine='xlsxwriter')
    df1.to_excel(writer, sheet_name='FY17 ACCOUNTS', columns=existing_cols)
    df2.to_excel(writer, sheet_name='NEW ACCOUNTS', columns=new_cols)
    writer.save()
    print "Validated data has been written in Excel!"

def get_overlapped_file_names(row_no, df_full_dup):
    temp_array = []
    for index, rows in df_full_dup.iterrows():
        if row_no == rows['row']:
            temp_array.append(rows['updated_file_name'])
    return "/".join(temp_array)

def get_overlapped_user_ids(row_no, df_full_dup):
    temp_array = []
    for index, rows in df_full_dup.iterrows():
        if row_no == rows['row']:
            temp_array.append(rows['fy17 owner userid '])
    return "/".join(temp_array)


def revalidate_by_overlap():
    xl = pd.ExcelFile('../account_assignment/assignment_fy17_accounts_validated.xlsx')
    df1 = xl.parse('FY17 ACCOUNTS')
    df2 = xl.parse('NEW ACCOUNTS')
    cols_rqrd1 = df1.columns
    cols_rqrd2 = df2.columns
    xl = pd.ExcelFile('../account_assignment/assignment_overlapping.xlsx')
    df_dup = xl.parse('overlap')
    print 'Re-validation on progress...'
    for index, rows in df_dup.iterrows():
        df1.ix[df1['row']==rows['row'], 'account type'] = np.nan
        df1.ix[df1['row']==rows['row'], 'fy17 owner userid '] = np.nan
        df1.ix[df1['row']==rows['row'], 'fy17 team agent name (for team agents only)'] = np.nan
        df1.ix[df1['row']==rows['row'], 'fy17 owner sl6 node'] = np.nan
        df1.ix[df1['row']==rows['row'], 'updated_file_name'] = np.nan
    print 'Re-validation completed!'
    print "Writing Re-validated data in Excel..."
    writer = pd.ExcelWriter('../account_assignment/assignment_fy17_accounts_re_validated.xlsx', engine='xlsxwriter')
    df1.to_excel(writer, sheet_name='FY17 ACCOUNTS', columns=cols_rqrd1)
    df2.to_excel(writer, sheet_name='NEW ACCOUNTS', columns=cols_rqrd2)
    writer.save()
    print "Re-validated data has been written in Excel!"
 
   
def write_overlap_in_excel():
    print 'Checking for overlapping....'
    xl = pd.ExcelFile('../account_assignment/assignment_fy17_accounts.xlsx')
    df = xl.parse('FY17 ACCOUNTS')
    cols_rqrd = df.columns
    cols_rqrd = cols_rqrd.append(pd.Series(['overlap_user_ids']))
    df_full_dup_rem = df.drop_duplicates(['row'], keep=False)
    df_half_dup_rem = df.drop_duplicates(['row'])
    total = len(df.index)
    no_dup_full = len(df_full_dup_rem.index)
    no_dup_half = len(df_half_dup_rem.index)

    df_full_dup = df[-df.row.isin(df_full_dup_rem.row)]
    df_half_dup = df_full_dup.drop_duplicates(['row'], keep='first')
    if len(df_full_dup.index) != 0:
        df_half_dup.updated_file_name = df_half_dup['row'].map(lambda x: get_overlapped_file_names(x, df_full_dup))
        df_half_dup['overlap_user_ids'] = df_half_dup['row'].map(lambda x: get_overlapped_user_ids(x, df_full_dup))

        print "Writing Overlapping data in Excel..."
        writer = pd.ExcelWriter('../account_assignment/assignment_overlapping.xlsx', engine='xlsxwriter')
        df_full_dup.to_excel(writer, sheet_name='full_overlap', columns=df.columns)
        df_half_dup.to_excel(writer, sheet_name='overlap', columns=cols_rqrd)
        writer.save()
        print "Overlapping data has been written in Excel!"
    else:
        print "No Duplicates Found!"
    print "Total Row(s): %d" % (total)
    print "Non Duplicated Row(s): %d (ful) | %d (half)" % (no_dup_full, no_dup_half)
    print "Remaining w.r.t Full Duplicates Row(s): %d" % (len(df_full_dup.index))
    print "Remaining w.r.t Half Duplicates Row(s): %d" % (len(df_half_dup.index))
    print 'Overlapping information has been written in Excel!'

    
    
#(df1, df2) = get_wb()
#print df1.columns
#(df_comp_existing, df_comp_new, df_log) = compile_data('compile_engine_input', df1.columns, df2.columns)
#validate_wb2(df1, df2, df_comp_existing, df_comp_new)
#write_overlap_in_excel()
#revalidate_by_overlap()
#print df2.columns

In [55]:
def get_input_revalidation():
    df_input = pd.read_excel('../account_assignment/validate_engine_input.xlsx')
    df_input = df_input[df_input.is_complete == 'No']
    df_inp_olap = df_input[(df_input.overlapping == 'Yes') & (df_input.revision == 'No')]
    df_inp_rev = df_input[(df_input.revision == 'Yes') & (df_input.overlapping == 'No')]
    return (df_inp_olap, df_inp_rev)
def get_versions(file_name, comment):
        xl = pd.ExcelFile('../account_assignment/'+file_name+'.xlsx')
        df_exist = xl.parse('existing')
        df_exist.rename(columns=string.lower, inplace=True)
        df_exist['file_version'] = comment
        df_newac = xl.parse('NEW ACCOUNTS')
        df_newac.rename(columns=string.lower, inplace=True)
        df_newac['file_version'] = comment
        return (df_exist, df_newac)
def get_wb():
    wd_path = '../account_assignment/'
    wb_file = wd_path + 'FY17_WB_v1_mapped.xlsx'
    xl_wb = pd.ExcelFile(wb_file)
    df1 = xl_wb.parse('FY17 ACCOUNTS')
    df1.rename(columns=string.lower, inplace=True)
    df2 = xl_wb.parse('NEW ACCOUNTS')
    df2.rename(columns=string.lower, inplace=True)
    return (df1, df2)

def get_overlapping():
    wd_path = '../account_assignment/'
    wb_file = wd_path + 'assignment_overlapping.xlsx'
    xl_wb = pd.ExcelFile(wb_file)
    df = xl_wb.parse('overlap')
    df.rename(columns=string.lower, inplace=True)
    return df

def write_wb(df1_1, df2_1, cols_rqrd1, cols_rqrd2, version):
    print "size of df1_1 is: %d" % (len(df1_1.index))
    print "Writing Re-validated data in Excel..."
    writer = pd.ExcelWriter('../account_assignment/assignment_fy17_accounts_re_validated_v'+str(version)+'.xlsx', engine='xlsxwriter')
    df1_1.to_excel(writer, sheet_name='FY17 ACCOUNTS', columns=cols_rqrd1)
    df2_1.to_excel(writer, sheet_name='NEW ACCOUNTS', columns=cols_rqrd2)
    writer.save()
    print "Re-validated data has been written in Excel!"

def validate_write_revision():
    print "Reading Inputs for the Revision Files..."
    (df_inp_olap, df_inp_rev) = get_input_revalidation()
    (df1, df2) = get_wb()
    cols_rqrd1 = df1.columns
    #print cols_rqrd1
    #cols_rqrd1 = cols_rqrd1.append(pd.Series(['updated_file_name','overlap_user_ids','overlap_update_info','remarks']))
    cols_rqrd2 = df2.columns
    df1_1 = pd.DataFrame(columns=cols_rqrd1)
    df2_1 = pd.DataFrame(columns=cols_rqrd2)
    print "Revision Files' details are in dataframe now!"
    print "Reading Revision Files & Validating..."
    for index, rows in df_inp_rev.iterrows():
        (df_old_exist, df_old_newac) = get_versions(rows.file_name_old, 'old')
        (df_new_exist, df_new_newac) = get_versions(rows.file_name_new, 'new')
        df_exist = pd.concat([df_old_exist, df_new_exist])
        df_newac = pd.concat([df_old_newac, df_new_newac])
        df_exist = df_exist.drop_duplicates(['row'], keep='last')
        df_newac = df_newac.drop_duplicates(['account name'], keep='last')
        df1 = pd.concat([df1, df_exist])
        df2 = pd.concat([df2, df_newac])
        df1 = df1.drop_duplicates(['row'], keep='last')
        #df2 = df2.drop_duplicates(['account name'], keep='last')
        df1_1 = pd.concat([df1_1, df1])
        df2_1 = pd.concat([df2_1, df2])
    print "Completed reading validating Revision Files!"
    print "Reading Overlapping Files & validating..."
    print "size of df1_1 is: %d" % (len(df1_1.index))
    for index, rows in df_inp_olap.iterrows():
        df_overlap = get_overlapping()
        df1_1 = pd.concat([df1_1, df_overlap])
        df1_1 = df1_1.drop_duplicates(['row'], keep='last')
    print "Completed reading & validating Overlapping Files!"
    print "size of df1_1 is: %d" % (len(df1_1.index))
    #New Accounts Addition in Rajeev's file
    print "Reading Rajeev's new Addition..."
    (df_rajeev_exist, df_rajeev_newac) = get_versions('Rajeev_FY17_WB_v1_SOUTH', 'new')
    df_rajeev_newac['updated_file_name'] = 'Rajeev_FY17_WB_v1_SOUTH'
    df2_1 = pd.concat([df2_1, df_rajeev_newac])
    print "Completed reading Rajeev's new Addition!"
    df1_1.ix[df1_1['account type'].str.contains('select', case=False).fillna(False),'account type'] = 'Named'
    df1_1.ix[df1_1['account type'].str.contains('mid', case=False).fillna(False),'account type'] = 'Named'
    df1_1.ix[df1_1['account type'].str.contains('delet', case=False).fillna(False),'account type'] = 'Named'
    write_wb(df1_1, df2_1, cols_rqrd1, cols_rqrd2, 2)

def write_overlap_in_excel():
    print 'Checking for overlapping....'
    xl = pd.ExcelFile('../account_assignment/assignment_fy17_accounts_re_validated_v2.xlsx')
    df = xl.parse('FY17 ACCOUNTS')
    cols_rqrd = df.columns
    cols_rqrd = cols_rqrd.append(pd.Series(['overlap_user_ids']))
    df_full_dup_rem = df.drop_duplicates(['row'], keep=False)
    df_half_dup_rem = df.drop_duplicates(['row'])
    total = len(df.index)
    no_dup_full = len(df_full_dup_rem.index)
    no_dup_half = len(df_half_dup_rem.index)

    df_full_dup = df[-df.row.isin(df_full_dup_rem.row)]
    df_half_dup = df_full_dup.drop_duplicates(['row'], keep='first')
    if len(df_full_dup.index) != 0:
        df_half_dup.updated_file_name = df_half_dup['row'].map(lambda x: get_overlapped_file_names(x, df_full_dup))
        df_half_dup['overlap_user_ids'] = df_half_dup['row'].map(lambda x: get_overlapped_user_ids(x, df_full_dup))

        print "Writing Overlapping data in Excel..."
        writer = pd.ExcelWriter('../account_assignment/assignment_overlapping.xlsx', engine='xlsxwriter')
        df_full_dup.to_excel(writer, sheet_name='full_overlap', columns=df.columns)
        df_half_dup.to_excel(writer, sheet_name='overlap', columns=cols_rqrd)
        writer.save()
        print "Overlapping data has been written in Excel!"
    else:
        print "No Duplicates Found!"
    print "Total Row(s): %d" % (total)
    print "Non Duplicated Row(s): %d (ful) | %d (half)" % (no_dup_full, no_dup_half)
    print "Remaining w.r.t Full Duplicates Row(s): %d" % (len(df_full_dup.index))
    print "Remaining w.r.t Half Duplicates Row(s): %d" % (len(df_half_dup.index))


def check_changes():
    xl = pd.ExcelFile('../account_assignment/FY17_WB_v1_mapped.xlsx')
    df1 = xl.parse('FY17 ACCOUNTS')
    df1.rename(columns=string.lower, inplace=True)
    xl = pd.ExcelFile('../account_assignment/FY17_WB_v1_mapped_v2.xlsx')
    df2 = xl.parse('FY17 ACCOUNTS')
    df2.rename(columns=string.lower, inplace=True)
    df_result = pd.merge(df2, df1, on='row')
    return df_result

#result = check_changes()
#pd.set_option('display.max_rows', 180)
#for x in result.columns:
#    print x
#result2 = result[pd.notnull(result['fy17 owner userid _x']) & ((result['fy17 owner userid _x'] != result['fy17 owner userid _y']) | (result['fy17 owner sl6 node_x'] != result['fy17 owner sl6 node_y']))]
#result2 = result2[['row','fy17 owner userid _x','fy17 team agent name (for team agents only)_x','fy17 owner sl6 node_x','fy17 owner userid _y','fy17 team agent name (for team agents only)_y','fy17 owner sl6 node_y']]
#writer = pd.ExcelWriter('../account_assignment/assignment_changes.xlsx', engine='xlsxwriter')
#result2.to_excel(writer, sheet_name='change')
#writer.save()
#print len(result2.index)
#validate_write_revision()
#write_overlap_in_excel()

In [21]:
def read_wb(file_name):
    xl = pd.ExcelFile('../account_assignment/'+file_name+'.xlsx')
    df = xl.parse('FY17 ACCOUNTS')
    return df

def check_split_counts(df):
    df_blank = df[pd.isnull(df['FY17 Owner Userid '])]
    df_nonblank = df[pd.notnull(df['FY17 Owner Userid '])]
    full_len = len(df.index)
    blank_len = len(df_blank.index)
    nonblank_len = len(df_nonblank.index)
    if (blank_len + nonblank_len) == full_len:
        print "Split Dataframes (%d + %d) row(s) are EQUAL to Full DataFrame (%d) row(s)" % (blank_len, nonblank_len, full_len)
    else:
        print "Split Dataframes (%d + %d) row(s) are NOT equal to Full DataFrame (%d) row(s)" % (blank_len, nonblank_len, full_len)
    return (df_blank, df_nonblank)
    

df_b = read_wb('FY17 Accounts WB_29th June_Bhavani')
#df_s = read_wb('Shivani_WB_Enterprise_29th June')
df_b['whose_file'] = 'bhavani'
print len(df_b.drop_duplicates(['ROW']).index)
#df_b.drop_duplicates(['ROW'], keep='last')
df_s['whose_file'] = 'shivani'
b_full_len = len(df_b.index)
s_full_len = len(df_s.index)
print "Master File contains %d Row(s) | Update File contains %d Row(s)" % (b_full_len, s_full_len)
(df_b_blank, df_b_nonblank) = check_split_counts(df_b)
(df_s_blank, df_s_nonblank) = check_split_counts(df_s)
df_b_final = pd.concat([df_b_blank, df_s_nonblank])
df_b_final = df_b_final.drop_duplicates(['ROW'], keep='last')
df_b = pd.concat([df_b_nonblank, df_b_final])
print len(df_b.drop_duplicates(['ROW'], keep='last').index)
#print len(df_b.index)


    

49610
Master File contains 49610 Row(s) | Update File contains 49610 Row(s)
Split Dataframes (7644 + 41966) row(s) are EQUAL to Full DataFrame (49610) row(s)
Split Dataframes (45519 + 4091) row(s) are EQUAL to Full DataFrame (49610) row(s)
49610


In [8]:
for element in df_b.columns:
    print element

ROW
Sales Level 2
Sales Level 3
Sales Level 4
Sales Level 5
Sales Level 6
Segment
Sub-Segment
AM/Team Agent Name
AM USERID
SAV ID
SAV Group Name
Super Group Flag
Party Id
Party Name
Party Type

HQ Party ID
Structured Account Template ID
Structure Account Template Name

Scope
Split Percent
DUNS no
SFDC Account Owner Userid
SFDC Account Name
SFDC Account ID
Line of Business
Address Line 1
City
State/Province
Country
Total no of Employees
No of PCs
No of Phones
FY16 Q4 Expected Product ($000)(Commit)
FY16 Q4 Expected Product ($000) (Upside)
FY16 Q4 Expected Service ($000) (Commit)
FY16 Q4 Expected Service ($000) (Upside)
FY17 Expected Product($000) (Commit)
FY17 Expected Product($000) (Upside)
FY14 Bookings (Product)
FY14 Bookings (Service)
FY15 Bookings (Product)
FY15 Bookings (Service)
FY16 M1-M10 Bookings (Product)
FY16 M1-M10 Bookings (Service)
Account Type
FY17 Owner Userid 
FY17 Team Agent Name (for Team Agents only)
FY17 OWNER SL6 NODE
FY17 SAV GROUP NAME
Sales Market Segment
Sub S